**MHNC LAB 09: REFORCEMENT LEARNING**

### NGUYỄN MINH HOÀNG 18110095

In [24]:

# tải các thư viện cần thiết


! pip install gym
!apt update
!apt install xvfb
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 $ibnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so

!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip install pyglet
!pip install pyopengl
!pip install pyvirtualdisplay



Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [25]:
%%bash

apt-get install -y xvfb x11-utils

pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
0 upgraded, 0 newly installed, 0 to remove and 78 not upgraded.


In [26]:


import io
import time
import random
import math
import glob
import base64
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML
from IPython.display import clear_output
from IPython import display as ipythondisplay

# ------------------------------------------
from keras.models     import Sequential
from keras.layers     import Dense
from tensorflow.keras.optimizers import Adam
# ----------------------------------------------
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) 

## 1. Giải thích vì sao khi dùng Naive-Learning thì xe taxi chỉ đứng yên một chỗ?

Bản thân Agent đứng yên tại một chỗ không hẳn là do chúng không di chuyển , mà là trong quá trình khởi tạo Agent , chúng sẽ có xu hướng di chuyển về phía bức tường rồi tiếp tục đập đầu vào nó. Điều này làm chúng ta lầm tưởng Agent bị đứng yên. Việc này xảy ra là do đối với thuật toán **Naive Learning**, agent sẽ chọn hành động có reward tốt nhất mà không quan tâm các action trong tương lai.

Đồng thời ta cũng không thể nói rằng Agent đứng yên do các action đón khách và trả khách vì dối với trường hợp các action này, thì ta nhận thấy rằng việc thực hiện action này sai vị trí sẽ bị trừ điểm rất nặng. Do thuộc tính của **Naive Learning** là điểm bằng nhau thì Agent sẽ chọn ngẫu nhiên action dẫn đến việc Agent có khả năng chọn các action này sai vị trí.

## 2. Giải thích vì sao khi dùng Q-Leaning thì xe taxi có thể đón và trả khách được?

Đối vói **Q-learning**, thuật toán này có cải tiến hơn so với **Naive Learning** ở chỗ chúng có quan tâm tới reward trong tương lai, việc này khiến agent sau khi thực hiện 1 action thì chúng còn quan sát các action khác để coi reward như thế nào. Như vậy sau khi huấn luyện đủ lâu trong environment , thì agent thấy rằng việc đón khách và trả khách đúng chỗ đem lại cho nó rất nhiều reward nên chúng sẽ luôn cân nhắc thực hiện các action này.

## 3. Tìm hiểu một game khác trên OpenAI và thiết lập cho agent chơi được

Ta sẽ chọn trò **MountainCar** trong OpenAI. Trò chơi này sẽ bắt đầu với 1 agent là 1 chiếc xe ở dưới đáy của 1 thung lũng. Với bất kì một state nào thì agent sẽ có 3 action là đi qua trái , đi qua phải hay ngừng ở một vận tốc bất kì.

**State** :
      State của trò chơi này sẽ có 2 phần tử. Thứ nhất là vị trí của xe, giá trị này nằm trong khoảng [-1.2,0.6]. Thứ hai là vận tốc của xe và chúng nằm trong khoảng [-0.07,0.07].

**Action** : 
      Như đã đề cập ở trên sẽ có 3 action có giá trị là [0,1,2] và các hoạt động tương ứng của nó là: 
  0.   Di chuyển sang trái
  1.   Không di chuyển
  2.   Di chuyển sang phải

**Reward** : 
      Ta sẽ có reward là 0 nếu Agent đi tới cây cờ (vị trí 0.5) ở trên đỉnh núi. Ngược lại mỗi bước di chuyển của Agent sẽ bị trừ đi 1 điểm nếu không đến được lá cờ. **Ta có thể thấy rằng reward trong trò chơi này sẽ là số âm.**

**Starting State**:
      Vị trí bắt đầu của chiếc xe sẽ là một con số bất kì nằm trong khoảng [-0.6,-0.4]. Và vận tốc bắt đầu là 0.

**Episode Termination**:
      Trò chơi sẽ kết thúc nếu Agent qua được lá cờ (vị trí lớn hơn hay bằng 0.5) hay số lần di chuyển trong 1 episode vượt quá 200.





In [27]:
def show_video():
  '''
  Hàm này sẽ giúp ta xuất ra video ở episode cuối về cách chơi của Agent
  '''
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  '''
  Hàm này sẽ cho ta kích hoạt video
  Hàm có input là tên environment
  '''
  env = gym.wrappers.Monitor(env, './video'  , force=True)
  return env


def query_environment(name):
  '''
  Hàm này dùng để in ra thử các thuộc tính của môi trường như action, reward...
  '''
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

In [28]:
query_environment('MountainCar-v0')

Action Space: Discrete(3)
Observation Space: Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
Max Episode Steps: 200
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: -110.0


In [29]:
from pyvirtualdisplay import Display
# dùng để display từng frame của agent
display = Display(visible=0, size=(1400, 900))   
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [30]:
# env = gym.make('MountainCar-v0')
env = wrap_env(gym.make('MountainCar-v0'))
env.reset()

array([-0.5032905,  0.       ])

In [31]:
action = random.randrange(0, 3)
state, reward, done, info = env.step(action)
print(state)

[-5.03442721e-01 -1.52217836e-04]


In [33]:
n_action = env.action_space.n
env_low = env.observation_space.low
env_high = env.observation_space.high
bins = 30 # number of states for discretization
print('Number of actions possible: ', n_action)  # các action có thể thực hiện
print('Observation space: ', env.observation_space) # state của agent hiện tại (vị trí,vận tốc)
print('Highest value: ', env_high) # giá trị lớn nhất của vị trí và vận tốc
print('Lowest value: ', env_low) # giá trị nhỏ nhất của vị trí và vận tốc

Number of actions possible:  3
Observation space:  Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
Highest value:  [0.6  0.07]
Lowest value:  [-1.2  -0.07]


In [34]:

goal_steps = 200   # là giá trị tới hạn của trò chơi , vượt quá 200 trò chơi sẽ ngừng
score_requirement = -198 # số nhỏ hơn giá trị này cho biết agent đã vượt qua được lá cờ (do < 200)
intial_games = 10000  # số game mà ta muốn chơi

### a. RANDOM ACTION

In [41]:
episodes = 100
env = wrap_env(gym.make('MountainCar-v0'))
for i in range(episodes):  
  env.reset()
  rewards = 0
  for step_index in range(goal_steps):
      env.render()
      action = env.action_space.sample()  # chọn action bất kì
      observation, reward, done, info = env.step(action)
      rewards += reward
      if done:
          break
  print('Episode : {}, Reward : {}'.format(i+1,rewards))
  

Episode : 1, Reward : -200.0
Episode : 2, Reward : -200.0
Episode : 3, Reward : -200.0
Episode : 4, Reward : -200.0
Episode : 5, Reward : -200.0
Episode : 6, Reward : -200.0
Episode : 7, Reward : -200.0
Episode : 8, Reward : -200.0
Episode : 9, Reward : -200.0
Episode : 10, Reward : -200.0
Episode : 11, Reward : -200.0
Episode : 12, Reward : -200.0
Episode : 13, Reward : -200.0
Episode : 14, Reward : -200.0
Episode : 15, Reward : -200.0
Episode : 16, Reward : -200.0
Episode : 17, Reward : -200.0
Episode : 18, Reward : -200.0
Episode : 19, Reward : -200.0
Episode : 20, Reward : -200.0
Episode : 21, Reward : -200.0
Episode : 22, Reward : -200.0
Episode : 23, Reward : -200.0
Episode : 24, Reward : -200.0
Episode : 25, Reward : -200.0
Episode : 26, Reward : -200.0
Episode : 27, Reward : -200.0
Episode : 28, Reward : -200.0
Episode : 29, Reward : -200.0
Episode : 30, Reward : -200.0
Episode : 31, Reward : -200.0
Episode : 32, Reward : -200.0
Episode : 33, Reward : -200.0
Episode : 34, Rewar

In [42]:

show_video() 

Ta thấy rằng việc chọn action bất kì sau mỗi timestep khiến Agent không hoàn thành được trò chơi.

### b. DEEP Q-LEARNING

Q -learning là một thuật toán học tăng cường không mô hình. Mục tiêu của Q-learning là học một chính sách, chính sách cho biết máy sẽ thực hiện hành động nào trong hoàn cảnh nào. Q-learning sẽ dùng Q-table để lưu trữ các giá trị của state-action và Q-value tương ứng với state action đó.


Tuy nhiên trong bài toán mà ta có nhiều trương hợp state-action value thì việc lưu trữ này rất tốn kém. Do đó ta mới nãy ra 1 cách cải tiến hơn Q-learning là Deep Q-learning. 

Deep Q-learning = Q-learning + Deep Neural Network

Deep Q-learning là 1 cách mà ta dùng mạng Neuron để lưu trữ các giá trị trong Q-table.

In [36]:
def data_preparation(episodes):
    '''
    Hàm này dùng để chuẩn bị data cho Neural Network để chúng lưu trữ giá trị của Q-table
    Input đầu vào của hàm là số episodes mà ta cho agent chơi
    Output của hàm là data gồm các giá trị của thuộc tính state-action và giá trị q tương ứng state-action đó
    '''
    # env = gym.make('MountainCar-v0')

    env = wrap_env(gym.make('MountainCar-v0'))
    env.reset()
    training_data = []
    accepted_scores = [] # list này dùng để lưu trữ các agent có reward >= -198 , tức là agent đã về đích thành công
    for episode in range(episodes):
        # chạy từng episodes
        score = 0   # score sau mỗi episode
        game_memory = [] # list dùng để lưu trữ state-action và q-value
        previous_state = [] # list dùng để gán các state
        for step_index in range(goal_steps):
            action = random.randrange(0, 3)
            state, reward, done, info = env.step(action)
            
            if len(previous_state) > 0:
                game_memory.append([previous_state, action])
            previous_state = state
            if state[0] > -0.2:
              # policy cho agent, nếu agent tiếp tục lên thung lũng , ta sẽ cộng điểm
                reward = 1
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            # việc chỉ cập nhật ep sau 100 lần khiến cho người xem nghĩ agent học không tốt, ta cần lưu các lần qua đích thành công của agent
            accepted_scores.append(score)

            for data in game_memory:
              # trong vòng lặp này ta sẽ biến đổi các giá trị của action là [0,1,2] thành các one-hot vector
                if data[1] == 1:
                    output = [0, 1, 0]
                elif data[1] == 0:
                    output = [1, 0, 0]
                elif data[1] == 2:
                    output = [0, 0, 1]
                training_data.append([data[0], output])
        
        env.reset()
        if episode % 100 == 0 :
          print('Episode : {}, Reward : {}'.format(episode,score))
    
    print('Good performance of Agent : ',accepted_scores)
    
    return training_data
training_data = data_preparation(episodes = 10000) # ta sẽ cho agent train với 10000 ep

Episode : 0, Reward : -200.0
Episode : 100, Reward : -200.0
Episode : 200, Reward : -200.0
Episode : 300, Reward : -200.0
Episode : 400, Reward : -200.0
Episode : 500, Reward : -200.0
Episode : 600, Reward : -200.0
Episode : 700, Reward : -200.0
Episode : 800, Reward : -200.0
Episode : 900, Reward : -200.0
Episode : 1000, Reward : -200.0
Episode : 1100, Reward : -200.0
Episode : 1200, Reward : -200.0
Episode : 1300, Reward : -200.0
Episode : 1400, Reward : -200.0
Episode : 1500, Reward : -200.0
Episode : 1600, Reward : -200.0
Episode : 1700, Reward : -200.0
Episode : 1800, Reward : -200.0
Episode : 1900, Reward : -200.0
Episode : 2000, Reward : -200.0
Episode : 2100, Reward : -200.0
Episode : 2200, Reward : -200.0
Episode : 2300, Reward : -200.0
Episode : 2400, Reward : -200.0
Episode : 2500, Reward : -200.0
Episode : 2600, Reward : -200.0
Episode : 2700, Reward : -200.0
Episode : 2800, Reward : -200.0
Episode : 2900, Reward : -200.0
Episode : 3000, Reward : -200.0
Episode : 3100, Rewa

In [37]:
def build_model(input_size, output_size):
    '''
    Hàm khởi tạo một Neural Network để lưu trữ giá trị của Q-table
    Hàm có 2 input đầu vào :
          input_size : là số thuộc tính của bảng Q-table 
          output_size : là số action của Agent
    Hàm có Output là Model Neural Network
    '''

    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model

def train_model(training_data):
    '''
    Hàm dùng để cho model ta vừa tạo ở trên học từ data của Q-table
    Hàm có input đầu vào là Q-table gồm 2 thuộc tính state-action và Q-value
    Hàm có output là model sau khi train xong.
    '''  
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=5)
    return model

trained_model = train_model(training_data)

Epoch 1/5
293/293 [==============================] - 1s 2ms/step - loss: 0.2272
Epoch 2/5
293/293 [==============================] - 1s 2ms/step - loss: 0.2220
Epoch 3/5
293/293 [==============================] - 1s 2ms/step - loss: 0.2211
Epoch 4/5
293/293 [==============================] - 1s 2ms/step - loss: 0.2209
Epoch 5/5
293/293 [==============================] - 1s 2ms/step - loss: 0.2201


In [38]:
# Xem agent học như thế nào sau khi dùng kết quả từ model
env = wrap_env(gym.make('MountainCar-v0'))
scores = []

# ta sẽ cho chạy 100 ep
for episode in range(100):
    score = 0

    prev_state = []
    env.reset()
    for step_index in range(goal_steps):
        env.render()
        if len(prev_state)==0:
            action = random.randrange(0,3)
        else:
            action = np.argmax(trained_model.predict(prev_state.reshape(-1, len(prev_state)))[0])
        
        new_state, reward, done, info = env.step(action)
        prev_state = new_state

        score += reward
        if done:
            break
    scores.append(score)
    

print(scores)
print('Average Score:',sum(scores)/len(scores))


[-165.0, -161.0, -160.0, -162.0, -161.0, -131.0, -200.0, -163.0, -159.0, -200.0, -161.0, -162.0, -126.0, -200.0, -130.0, -167.0, -200.0, -160.0, -135.0, -130.0, -127.0, -140.0, -125.0, -195.0, -159.0, -161.0, -159.0, -125.0, -164.0, -129.0, -200.0, -162.0, -161.0, -162.0, -163.0, -179.0, -200.0, -200.0, -200.0, -161.0, -137.0, -165.0, -138.0, -127.0, -161.0, -159.0, -130.0, -161.0, -161.0, -162.0, -161.0, -159.0, -135.0, -159.0, -200.0, -165.0, -130.0, -125.0, -129.0, -127.0, -167.0, -134.0, -138.0, -172.0, -129.0, -200.0, -165.0, -200.0, -161.0, -161.0, -159.0, -135.0, -167.0, -177.0, -141.0, -162.0, -161.0, -159.0, -138.0, -161.0, -162.0, -162.0, -134.0, -127.0, -165.0, -159.0, -129.0, -130.0, -127.0, -159.0, -130.0, -161.0, -159.0, -128.0, -200.0, -165.0, -159.0, -159.0, -200.0, -200.0]
Average Score: -157.98


In [40]:
show_video()

Agent hoàn thành tốt trò chơi với Deep Q-LEARNING.